In [ ]:
!pip install dwave-ocean-sdk yfinance pandas scikit-learn keras numpy plotly

In [6]:
import yfinance as yf
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense
from dwave.system import DWaveSampler, EmbeddingComposite
import plotly.graph_objects as go
from datetime import timedelta

# Loaded Secrets
from kaggle_secrets import UserSecretsClient

# Set up D-Wave API token
user_secrets = UserSecretsClient()
API_TOKEN = user_secrets.get_secret("dwave-api")

# Retrieve AAPL stock data
def get_stock_data(ticker):
    stock = yf.Ticker(ticker)
    data = stock.history(period="1y")
    return data[['Close']]

# Preprocess the data
def preprocess_data(data):
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_data = scaler.fit_transform(data)
    return scaled_data, scaler

# Create dataset for LSTM
def create_dataset(data, time_step=1):
    X, Y = [], []
    for i in range(len(data) - time_step - 1):
        a = data[i:(i + time_step), 0]
        X.append(a)
        Y.append(data[i + time_step, 0])
    return np.array(X), np.array(Y)

# Define LSTM model
def create_model(input_shape):
    model = Sequential()
    model.add(LSTM(50, return_sequences=True, input_shape=(input_shape, 1)))
    model.add(LSTM(50, return_sequences=False))
    model.add(Dense(25))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

# Optimize hyperparameters using D-Wave
def optimize_hyperparameters():
    sampler = EmbeddingComposite(DWaveSampler(token=API_TOKEN))
    
    # Define a simple QUBO problem for demonstration (real optimization would be more complex)
    Q = {(0, 0): 1, (1, 1): 1, (0, 1): -1}
    
    # Solve the QUBO problem
    response = sampler.sample_qubo(Q, num_reads=100)
    return response.first.sample

# Main function
def main():
    data = get_stock_data('AAPL')
    last_3_months = data.index[-3*30:]  # Last 3 months data
    scaled_data, scaler = preprocess_data(data)
    
    time_step = 60
    X, Y = create_dataset(scaled_data, time_step)
    
    X = X.reshape(X.shape[0], X.shape[1], 1)
    
    model = create_model(time_step)
    
    # Optimize hyperparameters (if needed)
    optimized_params = optimize_hyperparameters()
    print("Optimized Parameters:", optimized_params)
    
    model.fit(X, Y, epochs=10, batch_size=32, verbose=2)
    
    # Predict the next 7 days
    predictions = []
    for _ in range(7):
        last_60_days = scaled_data[-time_step:]
        last_60_days = last_60_days.reshape(1, time_step, 1)
        pred_price = model.predict(last_60_days)
        predictions.append(pred_price[0, 0])
        scaled_data = np.append(scaled_data, pred_price)
    
    predictions = scaler.inverse_transform(np.array(predictions).reshape(-1, 1))
    print("Predicted Prices for the next 7 days:", predictions)

    # Create dates for the next 7 days
    last_date = data.index[-1]
    prediction_dates = [last_date + timedelta(days=i) for i in range(1, 8)]
    
    # Plot the results
    fig = go.Figure()
    
    fig.add_trace(go.Scatter(x=data.index[-90:], y=data['Close'][-90:], mode='lines', name='Historical Prices'))
    fig.add_trace(go.Scatter(x=prediction_dates, y=predictions.flatten(), mode='lines', name='Predicted Prices'))
    
    fig.update_layout(
        title='AAPL Stock Price Prediction',
        xaxis_title='Date',
        yaxis_title='Price',
        xaxis=dict(
            range=[last_3_months[0], prediction_dates[-1]]
        )
    )
    
    fig.show()

if __name__ == "__main__":
    main()


/opt/conda/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Optimized Parameters: {0: 0, 1: 0}
Epoch 1/10
6/6 - 4s - 601ms/step - loss: 0.1097
Epoch 2/10
6/6 - 0s - 44ms/step - loss: 0.0330
Epoch 3/10
6/6 - 0s - 44ms/step - loss: 0.0225
Epoch 4/10
6/6 - 0s - 48ms/step - loss: 0.0215
Epoch 5/10
6/6 - 0s - 50ms/step - loss: 0.0176
Epoch 6/10
6/6 - 0s - 48ms/step - loss: 0.0168
Epoch 7/10
6/6 - 0s - 51ms/step - loss: 0.0147
Epoch 8/10
6/6 - 0s - 48ms/step - loss: 0.0135
Epoch 9/10
6/6 - 0s - 48ms/step - loss: 0.0123
Epoch 10/10
6/6 - 0s - 43ms/step - loss: 0.0112
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 302ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
Predicted Prices for the next 7 days: [[208.32777]
 [209.1837 ]
 [209.76253]
 [210.14223]
 [210.37682]
 [210.50331]
 [210.54753]]


# Code Iteration .1

Improving the accuracy of a stock price prediction model to over 90% is a challenging task due to the inherent volatility and complexity of financial markets. However, there are several strategies and techniques you can use to enhance your model's performance:

1. Data Enhancement
Feature Engineering: Create additional features that might help the model. These could include:
Moving averages (e.g., 5-day, 20-day)
Volume changes
Technical indicators (e.g., RSI, MACD, Bollinger Bands)
Sentiment analysis from news or social media
Longer Historical Data: Use more historical data to capture long-term trends and patterns.
Multiple Data Sources: Combine data from different sources, such as economic indicators, competitor performance, and global market trends.
2. Advanced Models
Hybrid Models: Combine different types of models. For example, use an LSTM to capture temporal dependencies and a Transformer model to capture long-range dependencies.
Ensemble Methods: Use ensemble techniques like stacking, bagging, or boosting to combine the predictions of multiple models.
State-of-the-Art Architectures: Experiment with advanced architectures like Attention Mechanisms and Transformers.
3. Hyperparameter Tuning
Use more sophisticated hyperparameter optimization techniques such as Bayesian Optimization, Genetic Algorithms, or Grid Search.
Optimize more parameters and model architecture components, not just the LSTM layers.
4. Regularization and Robustness
Regularization Techniques: Apply techniques like dropout, weight regularization, and batch normalization to prevent overfitting.
Cross-Validation: Use k-fold cross-validation to ensure your model generalizes well to unseen data.
5. Alternative Loss Functions
Experiment with different loss functions that might be more appropriate for financial time series forecasting, such as Huber loss or Quantile loss.
6. Incorporate D-Wave More Effectively
Optimize different parts of the model or hyperparameters using D-Wave's quantum annealing.
Consider using D-Wave for portfolio optimization or other financial optimization problems that complement the forecasting model.
7. Experimentation and Iteration
Keep iterating and experimenting with different models, features, and techniques.
Perform thorough backtesting to ensure your model performs well in different market conditions.

In [18]:
import yfinance as yf
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense
from dwave.system import DWaveSampler, EmbeddingComposite
import plotly.graph_objects as go
from datetime import timedelta

# Loaded Secrets
from kaggle_secrets import UserSecretsClient

# Set up D-Wave API token
user_secrets = UserSecretsClient()
API_TOKEN = user_secrets.get_secret("dwave-api")

# Retrieve AAPL stock data including High, Low, Close
def get_stock_data(ticker):
    stock = yf.Ticker(ticker)
    data = stock.history(period="1y")
    return data[['High', 'Low', 'Close']]

# Calculate Central Pivot Range (CPR)
def calculate_cpr(data):
    high = data['High']
    low = data['Low']
    close = data['Close']
    pivot = (high + low + close) / 3
    r1 = 2 * pivot - low
    return r1.shift(-1)  # Tomorrow's R1

# Implement Range Filter Indicator
def range_filter(data, r1_value, sampling_period=192, range_multiplier=9):
    filtered_data = data.copy()
    for i in range(len(data) - sampling_period):
        if data['Close'][i] > r1_value * range_multiplier:
            filtered_data['Close'][i] = r1_value * range_multiplier
    return filtered_data

# Preprocess the data
def preprocess_data(data):
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_data = scaler.fit_transform(data)
    return scaled_data, scaler

# Create dataset for LSTM
def create_dataset(data, time_step=1):
    X, Y = [], []
    for i in range(len(data) - time_step - 1):
        a = data[i:(i + time_step), :]
        X.append(a)
        Y.append(data[i + time_step, 0])  # Assuming we predict based on Close price
    return np.array(X), np.array(Y)

# Define LSTM model
def create_model(input_shape):
    model = Sequential()
    model.add(LSTM(50, return_sequences=True, input_shape=(input_shape, 3)))  # 3 for High, Low, Close
    model.add(LSTM(50, return_sequences=False))
    model.add(Dense(25))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

# Optimize hyperparameters using D-Wave
def optimize_hyperparameters():
    sampler = EmbeddingComposite(DWaveSampler(token=API_TOKEN))
    
    # Define a simple QUBO problem for demonstration (real optimization would be more complex)
    Q = {(0, 0): 1, (1, 1): 1, (0, 1): -1}
    
    # Solve the QUBO problem
    response = sampler.sample_qubo(Q, num_reads=100)
    return response.first.sample

# Main function
def main():
    data = get_stock_data('AAPL')
    
    # Calculate CPR and get Tomorrow's R1
    r1 = calculate_cpr(data)
    r1_value = r1.iloc[-1]  # Tomorrow's R1 value
    
    # Apply Range Filter indicator
    filtered_data = range_filter(data, r1_value)
    
    # Preprocess the data
    scaled_data, scaler = preprocess_data(filtered_data[['High', 'Low', 'Close']])
    
    time_step = 60
    X, Y = create_dataset(scaled_data, time_step)
    
    X = X.reshape(X.shape[0], X.shape[1], 3)  # 3 for High, Low, Close
    
    model = create_model(time_step)
    
    # Optimize hyperparameters (if needed)
    optimized_params = optimize_hyperparameters()
    print("Optimized Parameters:", optimized_params)
    
    model.fit(X, Y, epochs=10, batch_size=32, verbose=2)
    
   # Predict the next 7 days
    predictions = []
    for _ in range(7):
        last_60_days = scaled_data[-time_step:]
        last_60_days = last_60_days.reshape(1, time_step, 1)
        pred_price = model.predict(last_60_days)
        predictions.append(pred_price[0, 0])
        scaled_data = np.append(scaled_data, pred_price)
    
    predictions = scaler.inverse_transform(np.array(predictions).reshape(-1, 1))
    print("Predicted Prices for the next 7 days:", predictions)

    # Create dates for the next 7 days
    last_date = data.index[-1]
    prediction_dates = [last_date + timedelta(days=i) for i in range(1, 8)]
    
    # Plot the results
    fig = go.Figure()
    
    fig.add_trace(go.Scatter(x=filtered_data.index[-90:], y=filtered_data['Close'][-90:], mode='lines', name='Filtered Prices'))
    fig.add_trace(go.Scatter(x=prediction_dates, y=predictions.flatten(), mode='lines', name='Predicted Prices'))
    
    fig.update_layout(
        title='AAPL Stock Price Prediction with Enhanced Model',
        xaxis_title='Date',
        yaxis_title='Price',
        xaxis=dict(
            range=[filtered_data.index[-90], prediction_dates[-1]]
        )
    )
    
    fig.show()

if __name__ == "__main__":
    main()


/tmp/ipykernel_33/367037676.py:37: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

/opt/conda/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



Optimized Parameters: {0: 0, 1: 0}
Epoch 1/10
6/6 - 4s - 607ms/step - loss: 0.0842
Epoch 2/10
6/6 - 0s - 48ms/step - loss: 0.0288
Epoch 3/10
6/6 - 0s - 51ms/step - loss: 0.0195
Epoch 4/10
6/6 - 0s - 47ms/step - loss: 0.0173
Epoch 5/10
6/6 - 0s - 45ms/step - loss: 0.0145
Epoch 6/10
6/6 - 0s - 45ms/step - loss: 0.0130
Epoch 7/10
6/6 - 0s - 44ms/step - loss: 0.0131
Epoch 8/10
6/6 - 0s - 50ms/step - loss: 0.0107
Epoch 9/10
6/6 - 0s - 48ms/step - loss: 0.0104
Epoch 10/10
6/6 - 0s - 47ms/step - loss: 0.0095


ValueError: cannot reshape array of size 180 into shape (1,60,1)